In [63]:
import argparse, textwrap, time, json, re, requests, random, datetime, copy
from datetime import datetime as dt
from copy import deepcopy

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from fake_useragent import UserAgent

global driver, params, AFTER_DATE ,BEFORE_DATE ,COURTESY_SLEEP ,PHOTOS_PER_PAGE ,VERBOSE ,TEST ,DUMP_PATH ,ADD_EXTRAS, HEADLESS

In [64]:
AFTER_DATE = dt.fromtimestamp(1072915200)
BEFORE_DATE = dt.now()
PHOTOS_PER_PAGE =  250
VERBOSE =  4
DUMP_PATH = './'
TEST = True
ADD_EXTRAS = 'url_o,original_format,date_taken,date_upload,geo'
HEADLESS = True
COURTESY_SLEEP = "0, 0.000000001"
GET_DATES_ONLY = True
BBOX_PHOTOS_PER_PAGE = 250

In [87]:
parser = argparse.ArgumentParser(description=textwrap.dedent('''\
        scrape JSONs containing photos from flickr
         '''))

parser.add_argument("-a", "--after_date"
                    , help="Start date (in unix timestamp format).\Defaults to yesterday"
                    , type=float, default=dt.now().timestamp() - (30 * 24 * 3600))
parser.add_argument("-b", "--before_date"
                    , help="End date (in unix timestamp format). Defaults to now", type=float
                    , default=dt.now().timestamp())
parser.add_argument("-s", "--courtesy_sleep"
                    , help="Range (in string format) from which a random value will be chosen to sleep randomly. example: '1.3, 2.7'", type=str, default="1.3, 2.7")
parser.add_argument("-n", "--photos_per_page"
                    , help="Photos per file. Default is 500 which is the maximum", type=int
                    , default=500)
parser.add_argument("-v", "--verbose"
                    , help="increase output verbosity", action="count", default=0)
parser.add_argument("-p", "--dump_path"
                    , help="Path where to dump json files", type=str, default='./')
parser.add_argument("-t", "--test"
                    , help="test mode", action='store_true')
parser.add_argument("-g", "--get_dates_only"
                    , help="Only scan for suitable ranges and store them on csv file.\
                    \n suitable ranges are dates where the photos returned aproximates\
                    4000", action='store_true')
parser.add_argument("-w", "--webdriver"
                    , help="Turn off headless mode for on the chrome webdriver"
                    , action='store_false')
parser.add_argument("-x", "--add_extras"
                    , help="extra json fields to request. Defaults to 'url_o,original_format\
                    ,date_taken,date_upload,geo'", type=str
                    , default="url_o,original_format,date_taken,date_upload,geo")

args = parser.parse_args()



args.__dict__['HEADLESS'] = args.webdriver

# ansigning global variables from cmdline args for easy typing
for each in args.__dict__: globals()[each.upper()] = args.__dict__[each]

# making sure the dates are in datetime format
for each in ["AFTER_DATE", "BEFORE_DATE"]:
    if type(each) != type(dt.now()):
        try:
            globals()[each] = dt.fromtimestamp(globals()[each])
        except Exception as e:
            print(f"please make sure the dates entered are in unix timestamps format: {e}")

# printing parameters for easy debugging
if VERBOSE >=3:
    print("".ljust(120, "_") + "\nscript parameters")
    for each in args.__dict__:
        print(f"{each.upper()}: {args.__dict__[each]}")
    print("".ljust(120, "_"))

usage: ipykernel_launcher.py [-h] [-a AFTER_DATE] [-b BEFORE_DATE]
                             [-s COURTESY_SLEEP] [-n PHOTOS_PER_PAGE] [-v]
                             [-p DUMP_PATH] [-t] [-g] [-w] [-x ADD_EXTRAS]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/hadakechi/Library/Jupyter/runtime/kernel-c908b222-326c-4671-b0a6-e9500f7e6224.json


SystemExit: 2

/Users/hadakechi/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3327: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [65]:
    
COURTESY_SLEEP = [float(COURTESY_SLEEP.split(',')[0]) , float(COURTESY_SLEEP.split(',')[1])]
if TEST and VERBOSE > 3: COURTESY_SLEEP = [0, 0.000000001]


DATA_PATH = ('./test/' if TEST else DUMP_PATH)

params = {
    #
    "bbox" : '-124.799423, 24.750821, -54.517891, 54.306268', 
    "sort" : "relevance",
    "parse_tags" : "1",
    "content_type" : "7",
    "lang": "en-US",
    "has_geo" :"1",
    "media" : "photos",
    "view_all" : "1",
    "text" : "clouds",
    "viewerNSID": "",
    "method" : "flickr.photos.search",
    "csrf" : "",
    "format" : "json",
    "hermes" : "1",
    "hermesClient" : "1",
    "nojsoncallback" : "1",
    "geo_context": '2', # 0: all , 1: indoors, 2 : outdoors
    "privacy_filter" : 1
}

privacy_filters = '''
"public photos" : '1',
"private photos visible to friends" : '2',
"private photos visible to family" : '3',
"private photos visible to friends & family": '4',
"completely private photos" : '5'
'''

FLICKR = 'https://flickr.com/search/'
TAGS = ['rain cloud', 'sun clouds', 'sunny clouds', 'clouds', 'cloud', 'sky', 'storm', 'weather', 'cloudy']

var_names = ["api_key", "reqId", "api_url", "extras"]
re_expressions = [r"(api_key)=([\dabcdef]*)(&)", r"(reqId)=([\dabcdef]*)(&)", r"(https:\/\/(\w+\.?)+(\/\w+)+)(\?)", r"extras=((\w+(%2)?)+?)?&"]
groups = [2,2,1,1]

variables = [dict(zip(["var_name", "regex", 'group'], each)) for each in [each for each in zip(var_names, re_expressions, groups)]]

In [66]:
def get_api_call_string():
    ''' use selenium to get the api_call string'''
    
    options = Options()

    if HEADLESS: options.add_argument('--headless')

    caps = DesiredCapabilities.CHROME
    caps['loggingPref'] = {'performance': 'ALL'}

    driver = webdriver.Chrome(options = options, desired_capabilities=caps)

    xhrCallIntercept_js = """
    (function(XHR) {
      "use strict";

      var element = document.createElement('div');
      element.id = "interceptedResponse";
      element.appendChild(document.createTextNode(""));
      document.body.appendChild(element);

      var open = XHR.prototype.open;
      var send = XHR.prototype.send;

      XHR.prototype.open = function(method, url, async, user, pass) {
        this._url = url; // want to track the url requested
        open.call(this, method, url, async, user, pass);
      };

      XHR.prototype.send = function(data) {
        var self = this;
        var oldOnReadyStateChange;
        var url = this._url;

        function onReadyStateChange() {
          if(self.status === 200 && self.readyState == 4 /* complete */) {
            document.getElementById("interceptedResponse").innerHTML +=
              '{"data":' + self._url + ', "headers" :' + self.headers + ' }*****';
          }
          if(oldOnReadyStateChange) {
            oldOnReadyStateChange();
          }
        }

        if(this.addEventListener) {
          this.addEventListener("readystatechange", onReadyStateChange,
            false);
        } else {
          oldOnReadyStateChange = this.onreadystatechange;
          this.onreadystatechange = onReadyStateChange;
        }
        send.call(this, data);
      }
    })(XMLHttpRequest);
    """

    try:
        url = FLICKR + "?has_geo=1&media=photos&view_all=1&text=" + TAGS[0]

        driver.get(url)

        driver.execute_script(xhrCallIntercept_js)

        if VERBOSE >=1: print('title : "{}"'.format(driver.title))

    except Exception as e:
        print('Error! Cannot open search page: ' + str(e))

    wait = False
    while wait != True:
        if VERBOSE >= 1: print("Getting AJAX data...")
        # trying scroll to trigger and api call
        try:
            if VERBOSE >= 3: print('attempting Scroll!')
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            if VERBOSE >= 3: print('Scrolled...')

            # waiting for the api call to be included in the DOM
            wait = WebDriverWait(driver, 15).until(EC.text_to_be_present_in_element((By.ID, "interceptedResponse"), "api_key"))

        except Exception as e:
            print("intercept failed!:" + str(e))

        intercepts = driver.find_elements_by_id('interceptedResponse')

    if wait == True and VERBOSE >= 1: print('ajax call intercepted!\n')
    xhr_api_call = intercepts[0].text

    cookies = driver.get_cookies()
    driver.close()
    
    return xhr_api_call, cookies

In [67]:
def parse_api_call(call_string):
    ''' parsing data from DOM element '''
    #creating variables for each parsed data
    for each in variables:

        if  re.search(each["regex"], string=call_string, flags=re.MULTILINE) != None:
            globals()[each["var_name"]] = re.search(each["regex"], string=call_string, flags=re.MULTILINE).group(each["group"])
        else:
            globals()[each["var_name"]] = None

    if VERBOSE >= 2:
        print("Extracted ajax params:--------\n")
        for each in var_names:
            print("%(var)s :     %(value)s" % {"var": each.ljust(10, ' '), "value" : globals()[each]})

In [85]:
def construct_date_ranges(path, params, session, start, stop, offset):
    
    '''From one big range construct a bunch of contiguous ones joined
    end to end containing aproximately 4000 photos each'''

    # copying request parameters to prepare to simultaneous execusion
    params_lcl = deepcopy(params)

    # initializing time index
    params_lcl['min_upload_date'] = stop
    
    ranges = ''
    time_to_break = False

    while True:
        if VERBOSE >= 3: print("".ljust(20, '+'))

        params_lcl['max_upload_date'] = params_lcl['min_upload_date']
        params_lcl['min_upload_date'] -= datetime.timedelta(days=offset)
        
        if params_lcl['min_upload_date'].timestamp() < start.timestamp():
            time_to_break = True
            params_lcl['min_upload_date'] = AFTER_DATE


        total_photos = s.get(api_url, params=params_lcl).json()['photos']['total']
        if VERBOSE >=3: print(f"New date range: {params_lcl['min_upload_date']} to \
        {params_lcl['max_upload_date']}______ total photos: {total_photos}")

        if not time_to_break:
            params_lcl['min_upload_date'], next_batch_size, offset =  find_best_date_range(
                session=s,params=params_lcl,
                start=params_lcl['min_upload_date'],
                stop=params_lcl['max_upload_date'],
                total_photos=total_photos,
                offset=offset)

        if VERBOSE >= 2: print(f"Next suitable range!: {params_lcl['min_upload_date']} to \
        {params_lcl['max_upload_date']}______ total photos : {next_batch_size}".ljust(120, ' '))
        
        ranges += f"\n{term.replace(' ','_')},{params_lcl['min_upload_date']},\
        {params_lcl['max_upload_date']},{next_batch_size}"
        
        if time_to_break: break
    
    try:
        with open(ranges_file, 'a') as outfile:
            outfile.write(ranges)
        if VERBOSE >= 3: print(f"{ranges_file} written succesfully!")
    except Exception as e:
        print(f"problem dumping json data: {str(e)}")


In [69]:
def looping_over_date_range(path, params, session, start, stop, offset):
    ''' loops over given date range'''

    # copying request parameters to prepare to simultaneous execusion
    params_lcl = deepcopy(params)

    last_response_time = 0

    # initializing time index
    params_lcl['min_upload_date'] = stop

    while params_lcl['min_upload_date'].timestamp() >= start.timestamp():
        if VERBOSE >= 3: print("".ljust(20, '+'))

        params_lcl['max_upload_date'] = params_lcl['min_upload_date']
        params_lcl['min_upload_date'] -= datetime.timedelta(days=offset)

        total_photos = s.get(api_url, params=params_lcl).json()['photos']['total']
        if VERBOSE >=3: print(f"New date range: {params_lcl['min_upload_date']} \
        to {params_lcl['max_upload_date']}______ total photos: {total_photos}")

        params_lcl['min_upload_date'], next_batch_size, offset = find_best_date_range(session=s
                                               ,params=params_lcl
                                               , start=params_lcl['min_upload_date']
                                               , stop=params_lcl['max_upload_date']
                                               , total_photos=total_photos
                                               , offset=offset)

        if VERBOSE >= 2: print(f"Next suitable range: {params_lcl['min_upload_date']} to {params_lcl['max_upload_date']}______ total photos : {next_batch_size}".ljust(120, ' '))

        if VERBOSE >= 1: print(f'starting JSON dump...')

        if TEST and VERBOSE > 3:
            print(" fake writing to file ")
        else:
            write_each_page_as_json_file(path=DATA_PATH, call_params=params_lcl, session=s)

        time.sleep(0.2)
        time.sleep(last_response_time * random.uniform(COURTESY_SLEEP[0], COURTESY_SLEEP[1]) * 2)

In [89]:
def find_best_date_range(session, params, start, stop, total_photos, offset):

    call_params = deepcopy(params)
    # creating local variables to avoid multiprocessing issues down the line
    call_params['per_page'] = 1
    call_params['extras'] = ''

    if VERBOSE >=3: print(f"Finding a better range (in 20 attempts or less) ...")
    repeats = 0

    # This loop check whether the returned total photos are just a hair under
    # which is the maximum allowed by the api. If not it cleverly adjusts the
    # offset. The adjustment value used is difference ratio to the wanted
    # number of photos (the assumption for this heuristic is that for a small
    # range the uploaded photos density will not change much. And thus the
    # difference % when applied to the range will give us a ballpark of the wanted range)

    while (
        int(total_photos) != 4000
        and (
            int(total_photos) > 4000
            or int(total_photos) < ( 4000 * 0.99 )
        )
        and not (
            int(total_photos) < 4000
            and repeats > 20
        )
    ):

        if int(total_photos) > 4000:
            if VERBOSE >=3: print(f"({str(repeats)}): too many    ({total_photos.ljust(5, '+')})\
            {start} --- {stop}", end = '\r')
            # here the % will be small because we overshot the value wanted
            try:
                offset = offset * 4000/ int(total_photos)
            except ZeroDivisionError:
                time_to_break = True
                pass
            

        if int(total_photos) <= 4000:

            if VERBOSE >=3: print(f"({str(repeats)}): not enough  ({total_photos.ljust(5, '-')})\
            {start} --- {stop}", end = '\r')
            # here the % will be big because we underestimated the range (plus a small nudge)
            try:
                offset = offset * 4000/ int(total_photos)
            except ZeroDivisionError:
                time_to_break = True
                pass
            
        #if TEST and VERBOSE > 3: print("\n", offset)
        start = stop - datetime.timedelta(offset)

        # shifting the date range
        call_params['min_upload_date'] = start
        total_photos = session.get(api_url, params=call_params).json()['photos']['total']

        repeats += 1
        if start <= AFTER_DATE:
            time_to_break = True
            call_params['min_upload_date'] = AFTER_DATE
            total_photos = session.get(api_url, params=call_params).json()['photos']['total']
            return start, total_photos, offset
        # if VERBOSE >3 and TEST: time.sleep(1)
    return start, total_photos, offset

In [71]:
def write_each_page_as_json_file(path, call_params, session):

    for each in added_params: call_params[each] = added_params[each]

    # making_sure the request pages are correct
    call_params['per_page'] = PHOTOS_PER_PAGE
    pages = s.get(api_url, params=call_params).json()['photos']['pages']
    
    
    for page in range(1, 1 + pages):
        print(f"Requesting page {page}...".ljust(120, ' '),  end='\r')

        call_params['page'] = page


        try:
            before = dt.now().timestamp()
            response = session.get(api_url, params=call_params)
            after = dt.now().timestamp()
            data = response.json()
            data['api_call_params'] = call_params
        except Exception as e:
            print(f"Couldn't request JSON data:____ {e}")

        last_response_time = after - before

        # Trying to write JSON data to file
        file_to_be_written = f"{path}{term.replace(' ','_')}_{str(call_params['min_upload_date'].timestamp())}-{str(call_params['max_upload_date'].timestamp())}_{page}.json"
        #if VERBOSE >=3: print(f" file path to be written: {file_to_be_written}\n\n")
        try:
            with open(file_to_be_written, 'w') as outfile:
                json.dump(data.json(), outfile)

            time_it_took = str(round(last_response_time, 2))
            print(f"{file_to_be_written} written succesfully! took {time_it_took} s")
            time.sleep(0.2)
        except Exception as e:
            print(f"problem dumping json data: {str(e)}")
        if VERBOSE >=2: print(f'sleeping for {last_response_time * random.uniform(COURTESY_SLEEP[0], COURTESY_SLEEP[1])} seconds... (for courtesy :P )'.ljust(120, ' '), end = '\r')
        time.sleep(last_response_time * random.uniform(COURTESY_SLEEP[0], COURTESY_SLEEP[1]))
    print("")

In [90]:
if __name__ == '__main__':

    xhr_api_call, cookies = get_api_call_string()

    parse_api_call(xhr_api_call)

    extras = extras.replace('%2C', ',')

    added_params = {
        "extras" : '',
        #"extras" : extras + ','+ ADD_EXTRAS,
        "per_page" : 1,
        "api_key" : api_key,
        "reqId" : reqId,
    }

    for each in added_params: params[each] = added_params[each]

    spoof_webdriver = False

    if spoof_webdriver: ua = UserAgent()

    with requests.sessions.Session() as s:
        for cookie in cookies:
            s.cookies.set(cookie['name'], cookie['value'])
        if spoof_webdriver: s.headers['User-Agent'] = str(ua.chrome)

    offset = 3    

    TEST_RANGE = TAGS[:-2] if TEST and VERBOSE > 3 else TAGS
    
    date_format = '%Y-%m-%d'
    ranges_file = f'./date_ranges_{AFTER_DATE}_{BEFORE_DATE.strftime(date_format)}.csv'
    
    try:
        with open(ranges_file, 'w') as outfile:
            outfile.write("Search_Term, Uploaded_After, Uploaded_Before, Batch_Size")
    except Exception as e:
        print(f"problem dumping json data: {str(e)}")
    
    for term in TEST_RANGE:

        params['text'] = term

        first_range = True

        start = AFTER_DATE
        stop = BEFORE_DATE

        params['min_upload_date'] = start
        params['max_upload_date'] = stop
        
        if GET_DATES_ONLY:
            construct_date_ranges(path=DATA_PATH, params=params, session=s
                                  , start=params['min_upload_date']
                                  , stop=params['max_upload_date']
                                  , offset = offset)
        else:
            looping_over_date_ranges(path=DATA_PATH, params=params, session=s, start=params['min_upload_date'], stop=params['max_upload_date'], offset = 3)
        if VERBOSE >=3:
            print("".ljust(120, "-"))
            print(f"Finished for term: {term}   in date range : from {start} to {stop}")
            print("".ljust(120, "-"))

        #time.sleep(random.uniform(COURTESY_SLEEP[0], COURTESY_SLEEP[1]) * 3)

title : "Search: rain cloud | Flickr"
Getting AJAX data...
attempting Scroll!
Scrolled...
ajax call intercepted!

Extracted ajax params:--------

api_key    :     fc3f615bbb8ca8e17f26b216ba1212c3
reqId      :     a35c06a1
api_url    :     https://api.flickr.com/services/rest
extras     :     can_comment%2Ccount_comments%2Ccount_faves%2Cdescription%2Cisfavorite%2Clicense%2Cmedia%2Cneeds_interstitial%2Cowner_name%2Cpath_alias%2Crealname%2Crotation%2Curl_sq%2Curl_q%2Curl_t%2Curl_s%2Curl_n%2Curl_w%2Curl_m%2Curl_z%2Curl_c%2Curl_l
++++++++++++++++++++
New date range: 2019-12-27 00:58:06.272516 to         2019-12-30 00:58:06.272516______ total photos: 10
Finding a better range (in 20 attempts or less) ...
Next suitable range!: 2017-07-25 19:06:02.248441 to         2019-12-30 00:58:06.272516______ total photos : 3990        
++++++++++++++++++++
New date range: 2015-02-19 13:13:58.224366 to         2017-07-25 19:06:02.248441______ total photos: 5953
Finding a better range (in 20 attempts or le

1577682557.453262